In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
from torchvision import models 

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
transform_test = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

In [4]:
num_classes = 2
classes=['Cat','Dog']

In [5]:
file='resnet_1.pth'

In [6]:
loaded_model=models.resnet18(pretrained=True)

In [7]:
for param in loaded_model.parameters():
    param.requires_grad=False

In [8]:
in_features=loaded_model.fc.in_features
loaded_model.fc=nn.Sequential(nn.Linear(in_features,64),
                        torch.nn.Dropout(0.3),
                        nn.LeakyReLU(),
                        nn.Linear(64,2)
                       )

In [9]:
loaded_model.load_state_dict(torch.load(file))
loaded_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
loaded_model=loaded_model.to(device)

In [11]:
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from PIL import ImageTk, Image
filename=None
img=None

def upload():
    global filename,img,root,img2,uploaded
    #filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("png files","*.png"),("all files","*.*")))#filetypes = (("jpeg files","*.jpg"),("all files","*.*"))
    filename=filedialog.askopenfilename(filetypes=[("Image File",'.jpg')])
    #print (filename)
    img=Image.open(filename)
    img=img.resize((250,250))
    img = ImageTk.PhotoImage(img)
    label_1=Label(root,image=img)
    label_1.place(x=24,y=14,width="250",height="250")
fnt=('Arial',15,'bold')   

def predict():
    global filename
    from PIL import Image
    import torch.nn.functional as F
    img_path=filename
    img1=Image.open(img_path)
    img1_trans=transform_test(img1)
    img1_trans_reshaped=img1_trans.view(-1,3,224,224)
    out=loaded_model(img1_trans_reshaped.to(device))
    out_softmax=F.softmax(out,1)
    max_val,pred=torch.max(out_softmax.data,1)
    classes=['Cat','Dog']
    prediction=classes[pred.data.item()]
    txt="It's a "+prediction+'!!!'
    
    l=Label(root,font=fnt,text=txt,fg='Gray24',bg='chartreuse2',relief=SUNKEN)
    l.place(x=50,y=425,width=188,height=45)

root = Tk()
root.configure(background='white')
root.title("Cat-Dog Predictor")
root.iconbitmap("cat_dog_icon.ico")
root.geometry("300x500")
f=Frame(root,width=300,height=500,bg='red')    
f.pack()
x=Button(root,cursor='hand2',text="Upload Image",bg="gold",fg="Gray24",width=15,height=1,font=("open sans",14,'bold'),relief=RAISED,command=upload)
x.place(x=50,y=300)
y=Button(root,cursor='hand2',text="Predict",bg="gold",fg="Gray24",width=15,height=1,font=("open sans",14,'bold'),relief=RAISED,command=predict)
y.place(x=50,y=350)
label1=Label(root,cursor='xterm',bg='turquoise1',relief=SUNKEN,text='Input Image')
label1.place(x=25,y=15,width="250",height="250")
root.mainloop()